In [1]:
import tabula
import numpy as np
import pandas as pd
from astropy.table import QTable
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.visualization import quantity_support
from astropy.io.ascii import write
from copy import deepcopy

# Download PDF

In [2]:
import requests

url = "https://arxiv.org/pdf/1409.5594.pdf"
filename = url.split('/')[-1]
response = requests.get(url)

with open(filename, "wb") as f:
    f.write(response.content)


# Read tables from PDF

In [3]:
tables = tabula.read_pdf("1409.5594.pdf", pages=21)

# Process tables

In [4]:
def split_err_cols(df):
    """
    inplace
    """
    for col in df.columns:
        if df[col].dtype == 'object' and df[col].str.contains(' ± ').any():
            df[col] = df[col].str.replace('−', '-')
            df[[col, f'{col}_err']] = df[col].str.split(' ± ', expand=True)
            df[col] = pd.to_numeric(df[col])
            df[f'{col}_err'] = pd.to_numeric(df[f'{col}_err'])
            
def split_err_cols_asym(df):
    """
    inplace
    """
    for col in df.columns:
        if df[col].dtype == 'object' and df[col].str.contains('\+').any():
            df[col] = df[col].str.replace('−', '-')
            df[[col, f'{col}_err_max']] = df[col].str.split('+', expand=True)
            df[[f'{col}_err_max', f'{col}_err_min']] = df[f'{col}_err_max'].str.split('-', expand=True)
            df[col] = pd.to_numeric(df[col])
            df[f'{col}_err_min'] = pd.to_numeric(df[f'{col}_err_min'])
            df[f'{col}_err_max'] = pd.to_numeric(df[f'{col}_err_max'])

            
def split_column(df, col_name):
    # Split the column into two
    dfs = df[col_name].str.split('-', expand=True)
    df1 = pd.to_numeric(dfs[0])
    df2 = pd.to_numeric(dfs[1])
    
    # Get the position of the original column
    col_index = df.columns.get_loc(col_name)

    # Drop the original column
    df = df.drop(columns=col_name)

    # Insert the new columns at the same position
    df.insert(col_index, f'{col_name}_max', df2)
    df.insert(col_index, f'{col_name}_min', df1)
    
    return df



def remove_units_labels(df):
    df.columns = df.columns.str.replace(r'\[.*?\]', '', regex=True)
    return df

def remove_spaces_from_labels(df):
    df.columns = df.columns.str.strip()
    return df


### Table A2

In [5]:
table_a2 = tabula.read_pdf("1409.5594.pdf", pages=19, multiple_tables=False, area=[0, 0, 29.9, 50], relative_area=True)[0]
remove_units_labels(table_a2)
table_a2 = remove_spaces_from_labels(table_a2)
table_a2 = split_column(table_a2, 'E')
split_err_cols(table_a2)
table_a2 = QTable.from_pandas(table_a2)
table_a2.add_row([11943, 18928, -6.7, 20.7, 24.4, 0.8, 0.9, 0.5])
table_a2['E_min'] = table_a2['E_min'] * u.GeV
table_a2['E_max'] = table_a2['E_max'] * u.GeV
for col in table_a2.columns:
    if 'E_' not in col:
        table_a2[col] /= 100
        table_a2[col].info.format = '.4f'
table_a2.write('magic_2015_table_a2.ecsv', format='ascii.ecsv', overwrite=True)
table_a2

E_min,E_max,bias,σ,RMS,bias_err,σ_err,RMS_err
GeV,GeV,,,,,,
float64,float64,float64,float64,float64,float64,float64,float64
47.0,75.0,0.2460,0.2180,0.2250,0.0090,0.0110,0.0040
75.0,119.0,0.0710,0.1980,0.2090,0.0040,0.0050,0.0020
119.0,189.0,-0.0010,0.1800,0.2130,0.0030,0.0030,0.0020
189.0,299.0,-0.0150,0.1680,0.2049,0.0030,0.0030,0.0018
299.0,475.0,-0.0220,0.1550,0.2020,0.0020,0.0020,0.0017
475.0,753.0,-0.0210,0.1480,0.2012,0.0020,0.0020,0.0018
753.0,1194.0,-0.0140,0.1540,0.2130,0.0020,0.0020,0.0020
1194.0,1892.0,-0.0180,0.1610,0.2130,0.0030,0.0030,0.0020


### Table A3

In [6]:
table_a3 = tabula.read_pdf("1409.5594.pdf", pages=19, multiple_tables=False, area=[0, 50, 30, 100], relative_area=True)[0]
remove_units_labels(table_a3)
table_a3 = remove_spaces_from_labels(table_a3)
table_a3 = split_column(table_a3, 'E')
split_err_cols(table_a3)
table_a3 = QTable.from_pandas(table_a3)
table_a3['E_min'] = table_a3['E_min'] * u.GeV
table_a3['E_max'] = table_a3['E_max'] * u.GeV
for col in table_a3.columns:
    if 'E_' not in col:
        table_a3[col] /= 100
        table_a3[col].info.format = '.4f'
table_a3.write('magic_2015_table_a3.ecsv', format='ascii.ecsv', overwrite=True)
table_a3

E_min,E_max,bias,σ,RMS,bias_err,σ_err,RMS_err
GeV,GeV,,,,,,
float64,float64,float64,float64,float64,float64,float64,float64
47.0,75.0,0.4580,0.2300,0.2660,0.0180,0.0200,0.0100
75.0,119.0,0.1890,0.2130,0.2370,0.0070,0.0080,0.0040
119.0,189.0,0.0380,0.1890,0.2220,0.0040,0.0040,0.0030
189.0,299.0,-0.0200,0.1820,0.2320,0.0030,0.0030,0.0020
299.0,475.0,-0.0380,0.1750,0.2230,0.0030,0.0030,0.0020
475.0,753.0,-0.0260,0.1680,0.2300,0.0030,0.0030,0.0020
753.0,1194.0,-0.0150,0.1660,0.2320,0.0030,0.0030,0.0020
1194.0,1892.0,-0.0040,0.1660,0.2450,0.0030,0.0030,0.0030


### Table A.4

Table A.4 is divided into two tables, corresponding to different zeniths

#### Zenith < 30deg

In [7]:
table_a4_zen_lower30 = tabula.read_pdf("1409.5594.pdf", pages=19, multiple_tables=False, area=[40, 50, 61, 69], relative_area=True,
                         pandas_options={'names':['E', 'theta_g']}
                          )[0]

split_err_cols(table_a4_zen_lower30)

table_a4_zen_lower30['theta_68'] = [0.157, 0.135, 0.108, 0.095, 0.081, 0.073, 0.071, 0.067, 0.065, 0.062, 0.056]
table_a4_zen_lower30['theta_68_err_min'] = [-0.007, -0.005, -0.003, -0.003, -0.003, -0.003, -0.003, -0.005, -0.004, -0.011, -0.012]
table_a4_zen_lower30['theta_68_err_max'] = [0.007, 0.005, 0.004, 0.004, 0.003, 0.004, 0.005, 0.006, 0.011, 0.012, 0.062]


table_a4_zen_lower30 = QTable.from_pandas(table_a4_zen_lower30)
table_a4_zen_lower30['E'] = table_a4_zen_lower30['E'] * u.GeV
for col in table_a4_zen_lower30.columns:
    if col.startswith('theta'):
        table_a4_zen_lower30[col] = table_a4_zen_lower30[col] * u.deg
    
table_a4_zen_lower30.write('magic_2015_table_a4_zen30.ecsv', format='ascii.ecsv', overwrite=True)

table_a4_zen_lower30


E,theta_g,theta_g_err,theta_68,theta_68_err_min,theta_68_err_max
GeV,deg,deg,deg,deg,deg
float64,float64,float64,float64,float64,float64
95.0,0.087,0.004,0.157,-0.007,0.007
150.0,0.075,0.002,0.135,-0.005,0.005
238.0,0.067,0.001,0.108,-0.003,0.004
378.0,0.058,0.001,0.095,-0.003,0.004
599.0,0.052,0.001,0.081,-0.003,0.003
949.0,0.046,0.001,0.073,-0.003,0.004
1504.0,0.044,0.001,0.071,-0.003,0.005
2383.0,0.042,0.002,0.067,-0.005,0.006


#### 30 deg < Zenith < 45 deg

In [8]:
table_a4_zen_30_45 = tabula.read_pdf("1409.5594.pdf", pages=19, multiple_tables=False, area=[40, 75, 61, 98], relative_area=True,
                         pandas_options={'names':['theta_g', 'theta_68']}
                          )[0]

split_err_cols(table_a4_zen_30_45)
split_err_cols_asym(table_a4_zen_30_45)
table_a4_zen_30_45 = QTable.from_pandas(table_a4_zen_30_45)
for col in table_a4_zen_30_45.columns:
    table_a4_zen_30_45[col] = table_a4_zen_30_45[col] * u.deg
table_a4_zen_30_45.add_column(table_a4_zen_lower30['E'], index=0)

table_a4_zen_30_45.write('magic_2015_table_a4_30zen45.ecsv', format='ascii.ecsv', overwrite=True)

table_a4_zen_30_45

E,theta_g,theta_68,theta_g_err,theta_68_err_max,theta_68_err_min
GeV,deg,deg,deg,deg,deg
float64,float64,float64,float64,float64,float64
95.0,0.088,0.129,0.013,0.009,0.021
150.0,0.078,0.148,0.005,0.017,0.013
238.0,0.072,0.12,0.003,0.009,0.007
378.0,0.063,0.097,0.003,0.008,0.006
599.0,0.054,0.083,0.003,0.007,0.007
949.0,0.052,0.082,0.002,0.006,0.005
1504.0,0.046,0.077,0.002,0.007,0.004
2383.0,0.045,0.068,0.003,0.01,0.006


In [9]:
def process_sensitivity_tables(df):
    for col in list(df.columns):
        if 'Nbkg' in col or 'Unnamed' in col:
            df = df.drop(columns=col)

    
    if len(df.columns) == 9:
        colnames = [
            'e_min',
            'e_max',
            'gamma-rate',
            'bkg-rate',
            'sensitivity_snr_cu',
            'sensitivity_lima_1off_cu',
            'sensitivity_lima_3off_cu',
            'sensitivity_lima_5off_cu',
            'sensitivity_snr',
        ]
    else:
        colnames = [
            'e_thresh',
            'gamma-rate',
            'bkg-rate',
            'sensitivity_snr_cu',
            'sensitivity_lima_1off_cu',
            'sensitivity_lima_3off_cu',
            'sensitivity_lima_5off_cu',
            'sensitivity_snr',
        ]


    df.columns = colnames

    df = df[1:]


    units = [
        u.GeV,
        u.GeV,
        1/u.min,
        1/u.min,
        u.one,
        u.one,
        u.one,
        u.one,
        1e-12/(u.cm**2*u.s*u.TeV)
    ]

    # split error columns
    split_err_cols(df)

    # convert to astropy table_a7le and set units

    df = QTable.from_pandas(df)
    for i,col in enumerate(df.columns):
        df[col] = (df[col].astype(float)) * (units+units[2:])[i]
        
    return df

In [10]:
tables = tabula.read_pdf("1409.5594.pdf", pages=20, )

Got stderr: juin 07, 2023 10:06:09 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
AVERTISSEMENT: No Unicode mapping for radicalbig (112) in font UWSBVB+txex



## Table A.5

In [11]:
table_a5 = process_sensitivity_tables(tabula.read_pdf("1409.5594.pdf", pages=20)[0])
table_a5.write('magic_2015_table_a5.ecsv', format='ascii.ecsv', overwrite=True)

table_a5

Got stderr: juin 07, 2023 10:06:12 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
AVERTISSEMENT: No Unicode mapping for radicalbig (112) in font UWSBVB+txex



e_thresh,gamma-rate,bkg-rate,sensitivity_snr_cu,sensitivity_lima_1off_cu,sensitivity_lima_3off_cu,sensitivity_lima_5off_cu,sensitivity_snr,gamma-rate_err,bkg-rate_err,sensitivity_snr_cu_err,sensitivity_lima_1off_cu_err,sensitivity_lima_3off_cu_err,sensitivity_lima_5off_cu_err,sensitivity_snr_err
GeV,GeV,1 / min,1 / min,,,,,1 / (cm2 s TeV),1 / min,1 / min,,,,
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
84.0,19.1,8.73,2.29,2.29,2.29,2.29,156.5,2e-13,0.07,0.03,0.03,0.03,0.03,1.7
86.0,18.8,7.8,2.07,2.07,2.07,2.07,137.1,2e-13,0.06,0.02,0.03,0.03,0.03,1.5
104.0,16.88,4.88,1.445,1.71,1.45,1.45,75.9,1.9e-13,0.05,0.015,0.02,0.02,0.02,0.8
146.0,6.17,0.32,0.84,1.25,1.0,0.95,28.6,1e-13,0.013,0.02,0.03,0.02,0.02,0.8
218.0,3.63,0.07,0.66,1.06,0.83,0.78,13.4,7e-14,0.006,0.03,0.04,0.03,0.03,0.7
289.0,2.94,0.032,0.56,0.93,0.72,0.67,7.6,7e-14,0.004,0.04,0.05,0.04,0.04,0.5
404.0,3.05,0.03,0.51,0.87,0.67,0.63,4.4,7e-14,0.004,0.04,0.04,0.04,0.03,0.3
523.0,2.51,0.023,0.55,0.95,0.72,0.68,3.2,6e-14,0.003,0.04,0.05,0.04,0.05,0.3


## Table A.6

In [12]:
table_a6 = tabula.read_pdf("1409.5594.pdf", pages=20)[1]
table_a6 = table_a6.drop(table_a6.columns[5], axis=1)
table_a6 = process_sensitivity_tables(table_a6)
table_a6.write('magic_2015_table_a6.ecsv', format='ascii.ecsv', overwrite=True)

table_a6

Got stderr: juin 07, 2023 10:06:14 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
AVERTISSEMENT: No Unicode mapping for radicalbig (112) in font UWSBVB+txex



e_thresh,gamma-rate,bkg-rate,sensitivity_snr_cu,sensitivity_lima_1off_cu,sensitivity_lima_3off_cu,sensitivity_lima_5off_cu,sensitivity_snr,gamma-rate_err,bkg-rate_err,sensitivity_snr_cu_err,sensitivity_lima_1off_cu_err,sensitivity_lima_3off_cu_err,sensitivity_lima_5off_cu_err,sensitivity_snr_err
GeV,GeV,1 / min,1 / min,,,,,1 / (cm2 s TeV),1 / min,1 / min,,,,
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
114.0,19.7,7.68,1.95,1.95,1.95,1.95,91.4,4e-13,0.1,0.03,0.04,0.04,0.04,1.6
119.0,19.3,6.83,1.76,1.77,1.76,1.76,78.4,3e-13,0.1,0.03,0.03,0.04,0.04,1.3
141.0,17.4,4.23,1.22,1.55,1.26,1.22,43.5,3e-13,0.08,0.02,0.03,0.03,0.03,0.8
210.0,5.6,0.216,0.76,1.15,0.92,0.86,16.1,1.6e-13,0.017,0.04,0.05,0.04,0.04,0.8
310.0,3.59,0.07,0.67,1.07,0.84,0.79,8.3,1.2e-13,0.01,0.05,0.07,0.06,0.06,0.7
401.0,3.19,0.051,0.65,1.05,0.82,0.77,5.6,1.2e-13,0.008,0.06,0.08,0.07,0.06,0.5
435.0,3.31,0.053,0.63,1.03,0.8,0.75,4.8,1.2e-13,0.009,0.06,0.08,0.07,0.06,0.4
546.0,2.98,0.042,0.63,1.03,0.8,0.75,3.4,1.1e-13,0.008,0.06,0.08,0.07,0.06,0.3


## Table A7

In [13]:
table_a7 = tabula.read_pdf("1409.5594.pdf", pages=21, area=[0, 0, 30, 100], relative_area=True,)[0]
table_a7 = process_sensitivity_tables(table_a7)
table_a7.write('magic_2015_table_a7.ecsv', format='ascii.ecsv', overwrite=True)

table_a7

e_min,e_max,gamma-rate,bkg-rate,sensitivity_snr_cu,sensitivity_lima_1off_cu,sensitivity_lima_3off_cu,sensitivity_lima_5off_cu,sensitivity_snr,gamma-rate_err,bkg-rate_err,sensitivity_snr_cu_err,sensitivity_lima_1off_cu_err,sensitivity_lima_3off_cu_err,sensitivity_lima_5off_cu_err,sensitivity_snr_err
GeV,GeV,1 / min,1 / min,,,,,1 / (cm2 s TeV),1 / min,1 / min,,,,,1 / (cm2 s TeV)
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
63.0,100.0,3.01,4.06,6.7,8.8,7.1,6.8,7.3e-10,0.13,0.08,0.2,0.4,0.3,0.3,3e-11
100.0,158.0,4.29,2.41,3.31,4.77,3.87,3.67,1.37e-10,0.12,0.06,0.12,0.14,0.11,0.1,5e-12
158.0,251.0,3.37,0.54,2.0,2.95,2.38,2.25,3.05e-11,0.08,0.03,0.08,0.1,0.08,0.08,1.3000000000000001e-12
251.0,398.0,1.36,0.066,1.72,2.8,2.16,2.03,9.300000000000001e-12,0.05,0.01,0.15,0.2,0.16,0.15,8e-13
398.0,631.0,1.22,0.027,1.23,2.1,1.61,1.51,2.3e-12,0.04,0.006,0.16,0.18,0.18,0.15,3e-13
631.0,1000.0,0.88,0.0133,1.19,2.18,1.64,1.53,7.2e-13,0.04,0.0018,0.1,0.12,0.09,0.11,6e-14
1000.0,1585.0,0.58,0.0059,1.21,2.48,1.8,1.66,2.3000000000000003e-13,0.03,0.0007,0.1,0.11,0.09,0.09,1.8e-14
1585.0,2512.0,0.3,0.0027,1.58,3.8,2.6,2.36,9e-14,0.02,0.0005,0.18,0.2,0.19,0.18,1e-14


## Table A8

In [14]:
table_a8 = tabula.read_pdf("1409.5594.pdf", pages=21, area=[37, 0, 55, 100], relative_area=True,)[0]
table_a8 = process_sensitivity_tables(table_a8)
table_a8.write('magic_2015_table_a8.ecsv', format='ascii.ecsv', overwrite=True)

table_a8

e_min,e_max,gamma-rate,bkg-rate,sensitivity_snr_cu,sensitivity_lima_1off_cu,sensitivity_lima_3off_cu,sensitivity_lima_5off_cu,sensitivity_snr,gamma-rate_err,bkg-rate_err,sensitivity_snr_cu_err,sensitivity_lima_1off_cu_err,sensitivity_lima_3off_cu_err,sensitivity_lima_5off_cu_err,sensitivity_snr_err
GeV,GeV,1 / min,1 / min,,,,,1 / (cm2 s TeV),1 / min,1 / min,,,,,1 / (cm2 s TeV)
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
100.0,158.0,3.18,2.89,4.9,7.0,5.7,5.4,2.02e-10,0.16,0.05,0.4,0.4,0.3,0.3,1.5e-11
158.0,251.0,2.67,0.54,2.52,3.7,3.0,2.8,3.8e-11,0.19,0.04,0.19,0.3,0.3,0.2,3e-12
251.0,398.0,2.86,0.305,1.76,2.64,2.11,2.0,9.5e-12,0.13,0.019,0.14,0.14,0.11,0.1,8e-13
398.0,631.0,1.76,0.088,1.5,2.41,1.9,1.79,2.7999999999999998e-12,0.12,0.006,0.2,0.16,0.14,0.13,4e-13
631.0,1000.0,1.44,0.038,1.23,2.04,1.58,1.48,7.4e-13,0.09,0.002,0.13,0.12,0.09,0.1,8e-14
1000.0,1585.0,0.94,0.0197,1.36,2.38,1.81,1.69,2.6e-13,0.08,0.0016,0.12,0.16,0.13,0.13,2e-14
1585.0,2512.0,0.67,0.0111,1.43,2.7,2.0,1.85,8.2e-14,0.06,0.0015,0.16,0.2,0.19,0.18,9e-15
2512.0,3981.0,0.32,0.0093,2.8,5.3,3.9,3.7,4.6e-14,0.05,0.0012,0.4,0.7,0.6,0.5,7e-15
